In [58]:
import pandas as pd
import warnings
from unidecode import unidecode
from sportsreference.ncaab.roster import Player
from re import search
from tqdm import tqdm, tqdm_notebook
import numpy as np
import time
warnings.filterwarnings('ignore')

In [59]:
from sportsreference.ncaab.roster import Player

In [60]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [61]:
df = pd.read_csv("NBA_data.csv")

In [62]:
#make all column titles lower case
df.columns = map(str.lower, df.columns)
df.head()

,win shares,mins played (per game),player,games,year,rank,points(per game),ws/48,box +/-,value overall replacement player,team,college,years,mins played (total),points (total),class
0,48.0,30.6,Kenyon Martin,757,2000,1,12.3,0.100,0.7,15.5,NJN,Cincinnati,15,23134,9325,Player
1,0.3,15.6,A.J. Guyton,80,2000,32,5.5,0.010,-5.4,-1.1,CHI,Indiana,3,1246,442,Player
2,11.6,14.3,Jake Voskuhl,450,2000,33,4.0,0.087,-2.2,-0.3,CHI,UConn,9,6443,1814,Player
3,0.6,18.7,Khalid El-Amin,50,2000,34,6.3,0.030,-3.8,-0.4,CHI,UConn,1,936,314,Bust
4,-0.1,10.6,Mike Smith,17,2000,35,3.0,-0.033,-5.5,-0.2,WAS,University of Louisiana at Monroe,1,180,51,Bust


In [63]:
def make_ids(col):
    global makes_ids
    clean_player = [unidecode(player) for player in col]
    suffix = "-1"
    makes_ids = [player.replace(".", "").replace("'", "").replace(" ", "-").lower()+suffix for player in clean_player]
    return makes_ids[:10]
    

In [64]:
make_ids(df["player"])

['kenyon-martin-1',
 'aj-guyton-1',
 'jake-voskuhl-1',
 'khalid-el-amin-1',
 'mike-smith-1',
 'eddie-house-1',
 'eduardo-najera-1',
 'lavor-postell-1',
 'hanno-mottola-1',
 'chris-carrawell-1']

In [ ]:
#see which players are generating errors when trying to call college data 
for player in tqdm_notebook(makes_ids):
    try:
        college = Player(player)
    except:
        print(f'Error for {player} in index # {makes_ids.index(player)}')
        


#exceptions
deeandre hullett, cc 23
kedrick brown, cc, 78
ronald murray, cc, 93
qyntel woods, cc, 128
jerome beasely, cc, 139
donta smith, cc, 172
luc-richard-mbah-a-moute-1, 337
bill-walker-1, 343
patrick-mills-1, 398
jeff-pendergraph-1, 403
bj-mullens-1, 424
bernard james, cc, 533
thomas-robinson-2, 555
ricky ledo, cc, 579
roy-devyn-marble-1, 644
kyle-anderson-3, 650
stephen-zimmermanjr-1, 720
kahlil-felder-1, 732
jakob-poeltl-1, 748
taurean-prince-1, 750
edrice-adebayo-1, 799
malik-milton-1, 830
raymond-spalding-1, 831
marvin-bagleyiii-1, 839
lonnie-walker-2, 853

In [65]:
#make list to fix incorrect player id's 
replace = [('luc-richard-mbah-a-moute-1', 337), ('bill-walker-1', 343), ('patrick-mills-1', 398), ('jeff-pendergraph-1', 403), 
          ('bj-mullens-1', 424),('thomas-robinson-2', 555), ('roy-devyn-marble-1', 644), ('kyle-anderson-3', 650), 
           ('stephen-zimmermanjr-1', 720), ('kahlil-felder-1', 732),('jakob-poeltl-1', 748), ('taurean-prince-1', 750),
           ('edrice-adebayo-1', 799), ('malik-milton-1', 830),('raymond-spalding-1', 831), ('marvin-bagleyiii-1', 839),
           ('lonnie-walker-2', 853)]

In [66]:
#replace incorrect id's
for player in replace:
    makes_ids[player[1]] = player[0]

In [67]:
#list players to remove
remove = ['deeandre-hulett-1', 'kedrick-brown-1', 'ronald-murray-1', 'qyntel-woods-1',
'jerome-beasley-1', 'donta-smith-1','bernard-james-1', 'ricky-ledo-1']
no_cc = [player for player in makes_ids if player not in remove] 
len(no_cc)

855

In [68]:
#list index of players to remove
remove_nums = [23, 78, 93, 128, 139, 172, 533, 579]

In [69]:
#remove same index rows from df
clean_df = df.drop(remove_nums)

In [70]:
#reset index
#re-index clean_df
clean_df.reset_index(drop=True, inplace=True)

In [71]:
#add Player_id column to clean NBA stats df
clean_df['player_id'] = no_cc

In [72]:
len(clean_df)

855

In [73]:
#run function that assigns schools to players and replaces hyphenated school names
def college_correct(col):
    global college_list
    college_list=[]
    for player in tqdm_notebook(col):
            try:
                college = Player(player).team_abbreviation
                college_list.append(college)
            except:
                print(f'Error for {player}')
    college_list = [school.replace("-", " ") for school in college_list]
    
        
        

In [74]:
#run function that assigns schools to players and replaces hyphenated school names
#we will use schools to check if we have the correct player_ids
college_correct(clean_df['player_id'])

In [75]:
#add to clean_df
clean_df['college_id'] = college_list

In [76]:
#create function to changes school names to popular abbreviations 
def college_abv(col):
    global col_list
    col_list = (col.replace("connecticut", "uconn").replace("louisiana state", "lsu")
              .replace("southern california", "usc") .replace('north carolina', 'unc').replace('southern methodist', 'smu')
              .replace('brigham young', 'byu').replace('nevada las vegas', 'unlv').replace('mississippi', 'ole miss')
              .replace('north carolina state', 'nc state').replace('massachusetts', 'umass').replace('pittsburgh', 'pitt')
              .replace('louisiana lafayette', 'la-lafayette'))
    

In [77]:
#run function and convert column
college_abv(clean_df["college_id"])
clean_df["college_id"] = col_list

In [78]:
#convert College column to lower case 
clean_df['college'] = clean_df['college'].str.lower() 

In [79]:
#create word column used for college matching
word = [words.split()[0] for words in clean_df['college_id']]
clean_df['word'] = word

In [80]:
#define function to see if college columns match.
#This alerts us to a possible incorrect player-id
def college_wrong(df):
    global wrong_players
    wrong_players = []
    for index, row in tqdm_notebook(df.iterrows()):
        time.sleep(.1)
        if search(row["word"], row["college"]):
            print(row["player"], "at ", index, " matches")
        else:
            print(row["player"], "at ", index, " does NOT match")
            wrong_players.append((row["player"]))
  
    

In [81]:
college_wrong(clean_df)

Kenyon Martin at  0  matches
A.J. Guyton at  1  matches
Jake Voskuhl at  2  matches
Khalid El-Amin at  3  matches
Mike Smith at  4  matches
Eddie House at  5  matches
Eduardo Nájera at  6  matches
Lavor Postell at  7  matches
Hanno Möttölä at  8  matches
Chris Carrawell at  9  matches
Michael Redd at  10  matches
Dan Langhi at  11  matches
Jabari Smith at  12  matches
Mark Karcher at  13  matches
Jason Hart at  14  matches
Kaniel Dickens at  15  matches
Ernest Brown at  16  does NOT match
Dan McClintock at  17  matches
Cory Hightower at  18  does NOT match
Chris Porter at  19  matches
Jaquay Walls at  20  matches
Scoonie Penn at  21  matches
Pete Mickeal at  22  matches
Brian Cardinal at  23  matches
Erick Barkley at  24  matches
Mark Madsen at  25  matches
Stromile Swift at  26  matches
Marcus Fizer at  27  matches
Mike Miller at  28  matches
DerMarr Johnson at  29  matches
Chris Mihm at  30  matches
Jamal Crawford at  31  matches
Keyon Dooling at  32  matches
Jérôme Moïso at  33  mat

Maurice Ager at  269  matches
Jordan Farmar at  270  matches
Kyle Lowry at  271  matches
Josh Boone at  272  matches
Shannon Brown at  273  matches
Rajon Rondo at  274  matches
Renaldo Balkman at  275  matches
Quincy Douby at  276  matches
Shawne Williams at  277  matches
Marcus Williams at  278  matches
Adam Haluska at  279  matches
Derrick Byars at  280  matches
Chris Richard at  281  matches
Marcus Williams at  282  does NOT match
Josh McRoberts at  283  matches
Jermareo Davidson at  284  matches
Glen Davis at  285  matches
Nick Fazekas at  286  matches
Reyshawn Terry at  287  matches
Jared Jordan at  288  matches
Taurean Green at  289  matches
Dominic McGuire at  290  matches
Aaron Gray at  291  matches
JamesOn Curry at  292  matches
Demetris Nichols at  293  matches
Herbert Hill at  294  matches
Ramon Sessions at  295  matches
Sammy Mejia at  296  matches
D.J. Strawberry at  297  matches
Gabe Pruitt at  298  matches
Stéphane Lasme at  299  matches
Carl Landry at  300  matches
Alan

Kim English at  531  matches
Quincy Miller at  532  matches
Justin Hamilton at  533  does NOT match
Kevin Murphy at  534  matches
Kyle O'Quinn at  535  matches
Kris Joseph at  536  matches
Darius Johnson-Odom at  537  matches
Robbie Hummel at  538  matches
Marcus Denmon at  539  matches
Darius Miller at  540  matches
Jeffery Taylor at  541  matches
Robert Sacre at  542  matches
Marquis Teague at  543  matches
Anthony Davis at  544  does NOT match
Michael Kidd-Gilchrist at  545  matches
Bradley Beal at  546  matches
Dion Waiters at  547  matches
Thomas Robinson at  548  matches
Damian Lillard at  549  matches
Harrison Barnes at  550  matches
Terrence Ross at  551  matches
Andre Drummond at  552  matches
Austin Rivers at  553  matches
Meyers Leonard at  554  matches
Jeremy Lamb at  555  matches
Kendall Marshall at  556  matches
John Henson at  557  matches
Festus Ezeli at  558  matches
Royce White at  559  matches
Perry Jones at  560  matches
Arnett Moultrie at  561  matches
Miles Plumle

Luke Kennard at  789  matches
Donovan Mitchell at  790  matches
Bam Adebayo at  791  matches
Justin Patton at  792  matches
Justin Jackson at  793  does NOT match
Tony Bradley at  794  matches
Kyle Kuzma at  795  matches
Caleb Swanigan at  796  matches
Tyler Lydon at  797  matches
OG Anunoby at  798  matches
Harry Giles at  799  matches
John Collins at  800  matches
T.J. Leaf at  801  matches
D.J. Wilson at  802  matches
Jarrett Allen at  803  matches
Justin Jackson at  804  does NOT match
Bruce Brown at  805  does NOT match
Jarred Vanderbilt at  806  matches
Jevon Carter at  807  matches
Gary Trent at  808  does NOT match
Melvin Frazier at  809  matches
Devonte' Graham at  810  matches
Jalen Brunson at  811  matches
Khyri Thomas at  812  matches
Thomas Welsh at  813  matches
De'Anthony Melton at  814  matches
Sviatoslav Mykhailiuk at  815  matches
Keita Bates-Diop at  816  matches
Chimezie Metu at  817  matches
Alize Johnson at  818  matches
Tony Carr at  819  matches
Vince Edwards at

In [82]:
len(wrong_players)

54

In [83]:
#check wrong players
select = clean_df["player"].isin(wrong_players)
wrong_df = clean_df[select]
wrong_df

,win shares,mins played (per game),player,games,year,rank,points(per game),ws/48,box +/-,value overall replacement player,team,college,years,mins played (total),points (total),class,player_id,college_id,word
16,-0.2,7.0,Ernest Brown,3,2000,52,1.0,-0.349,-24.7,-0.1,MIA,indian hills community college,1,21,3,Bust,ernest-brown-1,alabama,alabama
18,0.0,0.0,Cory Hightower,0,2000,54,0.0,0.000,0.0,0.0,SAS,indian hills community college,0,0,0,Bust,cory-hightower-1,presbyterian,presbyterian
60,0.0,9.8,Ken Johnson,16,2001,49,2.0,-0.002,-10.0,-0.3,MIA,ohio state,1,156,32,Bust,ken-johnson-1,michigan state,michigan
112,58.5,27.7,Mike Dunleavy,986,2002,3,11.2,0.103,0.3,15.9,GSW,duke,15,27340,11048,Player,mike-dunleavy-1,south carolina,south
162,0.0,0.0,David Young,0,2004,41,0.0,0.000,0.0,0.0,SEA,nc central,0,0,0,Bust,david-young-1,clemson,clemson
205,0.0,9.2,Robert Whaley,23,2005,51,2.1,0.004,-3.5,-0.1,UTA,walsh university,1,212,49,Bust,robert-whaley-1,cincinnati,cincinnati
240,1.1,9.5,Bobby Jones,91,2006,37,3.2,0.060,-3.5,-0.3,MIN,washington,2,867,288,Bust,bobby-jones-1,unc,unc
252,0.1,10.5,Dee Brown,68,2006,46,2.1,0.008,-4.1,-0.4,UTA,illinois,2,711,140,Bust,dee-brown-1,jacksonville,jacksonville
278,0.0,15.2,Marcus Williams,203,2006,22,5.6,-0.001,-4.6,-2.0,NJN,uconn,4,3095,1128,Player,marcus-williams-1,uconn,uconn
282,-0.1,3.0,Marcus Williams,13,2007,33,1.1,-0.084,-7.5,-0.1,SAS,arizona,2,39,14,Bust,marcus-williams-1,uconn,uconn


In [84]:
#create list of players to remove
remove2 = ['ernest-brown-1', 'cory-hightower-1', ]

In [85]:
#create list of player ids to update
replace = [('ken-johnson-1', 'ken-johnson-2'), ('mike-dunleavy-1', 'mike-dunleavy-2'), ('david-young-1','dave-young-1'),
          ('bobby-jones-1', 'bobby-jones-2'), ('dee-brown-1', 'dee-brown-2'), ('gerald-henderson-1', 'gerald-henderson-2'),
          ('james-johnson-1', 'james-johnson-2'), ('wesley-johnson-1', 'wes-johnson-1'),
          ('james-anderson-1', 'james-anderson-3'), ('derrick-williams-1', 'derrick-williams-2'),
          ('tristan-thompson-1','tristan-thompson-2'), ('jordan-hamilton-1', 'jordan-hamilton-2'),
          ('mike-scott-1', 'mike-scott-4'), ('justin-hamilton-1', 'justin-hamilton-2'), ('anthony-davis-1', 'anthony-davis-5'),
          ('jared-cunningham-1', 'jared-cunningham-2'), ('ray-mccallum-1', 'ray-mccallum-2'), ('glen-rice-1', 'glen-rice-jr-1'),
          ('ryan-kelly-1', 'ryan-kelly-3'), ('reggie-bullock-1', 'reggie-bullock-2'), ('tim-hardaway-1', 'tim-hardaway-jr-1'),
          ('glenn-robinson-1', 'glenn-robinson-2'),('joe-harris-1', 'joe-harris--1'),
          ('joe-young-1', 'joseph-young-1'),
          ('josh-richardson-1', 'josh-richardson-2'), ('anthony-brown-1', 'anthony-brown-5'), ('aaron-white-1','aaron-white-3'),
          ('devin-booker-1', 'devin-booker-2'), ('larry-nance-1', 'larry-nance-2'),
          ('demetrius-jackson-1', 'demetrius-jackson-2'), ('sterling-brown-1', 'sterling-brown-2'),
          ('frank-jackson-1', 'frank-jackson-2'), ('josh-jackson-1', 'josh-jackson-2'), ('dennis-smith-1', 'dennis-smithjr-1'),
          ('justin-jackson-1', 'justin-jackson-4'), ('justin-jackson-1', 'justin-jackson-5'),
          ('bruce-brown-1', 'bruce-brown-2'), ('gary-trent-1', 'gary-trentjr-1'), ('vince-edwards-1', 'vince-edwards-2'),
          ('troy-brown-1', 'troy-brown-5'), ('jaren-jackson-1', 'jaren-jacksonjr-1'), 
          ('wendell-carter-1', 'wendell-carterjr-1'),('jerome-robinson-1', 'jerome-robinson-2'), ('michael-porter-1', 'michael-porterjr-1'),
          ('robert-williams-1', 'robert-williams-3')
          ]

In [86]:
#update player ids in clean_df
for player in tqdm_notebook(replace):
    clean_df.replace(to_replace = player[0],
                    value = player[1], inplace = True)

In [87]:
#input justin jackson 5, marcus williams 2 and marcus thornton 3
clean_df.loc[282, 'player_id'] = 'marcus-williams-2'
clean_df.loc[664, 'player_id'] = 'marcus-thornton-3'
clean_df.loc[804, 'player_id'] = 'justin-jackson-5'

In [88]:
#check replacements
clean_df.iloc[[278, 282, 376, 664, 793, 804]]

,win shares,mins played (per game),player,games,year,rank,points(per game),ws/48,box +/-,value overall replacement player,team,college,years,mins played (total),points (total),class,player_id,college_id,word
278,0.0,15.2,Marcus Williams,203,2006,22,5.6,-0.001,-4.6,-2.0,NJN,uconn,4,3095,1128,Player,marcus-williams-1,uconn,uconn
282,-0.1,3.0,Marcus Williams,13,2007,33,1.1,-0.084,-7.5,-0.1,SAS,arizona,2,39,14,Bust,marcus-williams-2,uconn,uconn
376,20.0,23.4,Marcus Thornton,483,2009,43,11.9,0.085,-0.8,3.4,MIA,lsu,8,11299,5750,Player,marcus-thornton-1,lsu,lsu
664,0.0,0.0,Marcus Thornton,0,2015,45,0.0,0.000,0.0,0.0,BOS,college of william & mary,0,0,0,Bust,marcus-thornton-3,lsu,lsu
793,4.8,19.9,Justin Jackson,192,2017,15,6.7,0.060,-2.7,-0.7,POR,unc,3,3818,1289,Player,justin-jackson-4,tennessee,tennessee
804,0.0,0.0,Justin Jackson,0,2018,43,0.0,0.000,0.0,0.0,DEN,maryland,0,0,0,Bust,justin-jackson-5,tennessee,tennessee


In [89]:
#remove players not in NCAA
clean_df = clean_df[~clean_df['player_id'].isin(remove2)]

In [90]:
#re-index clean_df
clean_df.reset_index(drop=True, inplace=True)

In [91]:
len(clean_df)

853

In [92]:
#re-run college_correct function to assign correct college_id's to replaced players
college_correct(clean_df["player_id"])

In [198]:
#assign to clean_df
clean_df['college_id'] = college_list

In [202]:
clean_df.iloc[[276, 280, 374, 662, 791, 802]]

,win shares,mins played (per game),player,games,year,rank,points(per game),ws/48,box +/-,value overall replacement player,team,college,years,mins played (total),points (total),class,player_id,college_id,word
276,0.0,15.2,Marcus Williams,203,2006,22,5.6,-0.001,-4.6,-2.0,NJN,uconn,4,3095,1128,Player,marcus-williams-1,connecticut,uconn
280,-0.1,3.0,Marcus Williams,13,2007,33,1.1,-0.084,-7.5,-0.1,SAS,arizona,2,39,14,Bust,marcus-williams-2,arizona,uconn
374,20.0,23.4,Marcus Thornton,483,2009,43,11.9,0.085,-0.8,3.4,MIA,lsu,8,11299,5750,Player,marcus-thornton-1,louisiana state,lsu
662,0.0,0.0,Marcus Thornton,0,2015,45,0.0,0.000,0.0,0.0,BOS,college of william & mary,0,0,0,Bust,marcus-thornton-3,william mary,lsu
791,4.8,19.9,Justin Jackson,192,2017,15,6.7,0.060,-2.7,-0.7,POR,unc,3,3818,1289,Player,justin-jackson-4,north carolina,tennessee
802,0.0,0.0,Justin Jackson,0,2018,43,0.0,0.000,0.0,0.0,DEN,maryland,0,0,0,Bust,justin-jackson-5,maryland,tennessee


In [40]:
#create  list to hold dictionary data and list for desired ncaa stats
ncaa_data = []

stats = ['player_id','name','assist_percentage', 'assists', 'block_percentage','blocks','effective_field_goal_percentage',
        'field_goal_percentage', 'free_throw_attempt_rate', 'free_throw_percentage', 'minutes_played',
        'points', 'steal_percentage', 'steals', 'three_point_percentage', 'three_pointers', 'total_rebound_percentage',
         'total_rebounds', 'true_shooting_percentage', 'turnover_percentage','turnovers', 'usage_percentage', 'box_plus_minus', 'games_played', 'height', 'win_shares_per_40_minutes']



In [204]:
#insert player stats into ncaa_data
ncaa_data = []
for player in tqdm_notebook(clean_df["player_id"]):
    ncaa = Player(player)
    player_list = []
    player_dict = {}
    for stat in stats:
        try:
            var = eval(f'ncaa.{stat}')
            player_list.append((stat, var))
        except:
            var = np.nan
            print(player,stat, var)
            player_list.append((stat, var))
    player_dict.update(player_list)
    ncaa_data.append(player_dict)
    

In [241]:
#convert ncaa_data to df
ncaa_df = pd.DataFrame(ncaa_data)
ncaa_df.columns

Index(['assist_percentage', 'assists', 'block_percentage', 'blocks',
       'box_plus_minus', 'effective_field_goal_percentage',
       'field_goal_percentage', 'free_throw_attempt_rate',
       'free_throw_percentage', 'games_played', 'height', 'minutes_played',
       'name', 'player_id', 'points', 'steal_percentage', 'steals',
       'three_point_percentage', 'three_pointers', 'total_rebound_percentage',
       'total_rebounds', 'true_shooting_percentage', 'turnover_percentage',
       'turnovers', 'usage_percentage', 'win_shares_per_40_minutes'],
      dtype='object')

(ACC, Big
12, Big Ten, SEC, Pac 12, and Big East)

ACC = louisville, duke, florida state, virginia, syracuse, nc state, notre dame, pittsburgh, boston college, virginia tech,
clemson, georgia tech, unc, miami, wake forest

Big 12= baylor, kansas, west virginia, texas tech, oklahoma, texas, tcu, iowa state, kansas state, oklahoma state

SEC = auburn, kentucky, lsu, mississippi state, florida, south carolina, tennessee, alabama, texas a&m, arkansas, ole miss, 
missouri, georgia, vanderbilt

pac12 = colorado, oregon, arizona, arizona state, usc, ucla, stanford, utah, washington state, california, oregon state, washington

big east = seton hall, creighton, villanova, marquette, butler, providence, xavier, georgetown, st johns, depaul

big 10 = maryland, penn state, illinois, iowa, rutgers, michigan state, purdue, wisc, minnesota, ohio state, indiana, michigan, nebraska, northwestern

In [ ]:
schools = clean_df["college_id"].unique()
schools = schools.tolist()
schools

In [249]:
 big_school  = [
 'indiana',
 'connecticut',
 'arizona state',
 'oklahoma',
 'st johns ny',
 'utah',
 'duke',
 'ohio state',
 'vanderbilt',
 'louisiana state',
 'syracuse',
 'auburn',
 'colorado',
 'purdue',
 'stanford',
 'iowa state',
 'florida',
 'texas',
 'michigan',
 'missouri',
 'ucla',
 'michigan state',
 'minnesota',
 'georgia tech',
 'kentucky',
 'oklahoma state',
 'arizona',
 'southern california',
 'maryland',
 'clemson',
 'kansas',
 'oregon',
 'georgetown',
 'california',
 'seton hall',
 'arkansas',
 'notre dame',
 'alabama',
 'north carolina',
 'villanova',
 'illinois',
 'tennessee',
 'wake forest',
 'virginia',
 'miami fl',
 'mississippi state',
 'depaul'
 'creighton',
 'marquette',
 'georgia',
 'louisville',
 'xavier',
 'boston college',
 'mississippi',
 'texas tech',
 'florida state',
 'providence',
 'wisconsin',
 'pittsburgh',
 'texas am',
 'washington',
 'north carolina state',
 'south carolina',
 'rutgers',
 'iowa',
 'washington state',
 'kansas state',
 'virginia tech',
 'west virginia',
 'baylor',
 'butler',
 'oregon state',
 'penn state',
 'boise state']


In [251]:
school_code =[]
for school in clean_df['college_id']:
    if school in big_school:
        school_num = 1
    else:
        school_num = 0
    school_code.append(school_num)
        

In [253]:
len(school_code)

853

In [254]:
ncaa_df['big_conf_flag'] = school_

In [255]:
#create new dataframe which re-orders columns and drops those with a lot of NaN's
ncaa2_df = ncaa_df[["player_id", "name",  "assists", "blocks", "effective_field_goal_percentage",
                    "field_goal_percentage", "free_throw_attempt_rate","free_throw_percentage", "games_played", "height",
                    "minutes_played", "points", "big_conf_flag", "steals", "three_point_percentage", "three_pointers", "total_rebounds",
                    "true_shooting_percentage", "turnover_percentage", "turnovers", "win_shares_per_40_minutes" ]]

In [207]:
ncaa2_df.fillna(ncaa2_df.mean(), inplace=True)

In [50]:
#stats per 40  list
per_40 = [ "assists", "blocks", "points", "steals", "three_pointers", "total_rebounds", "turnovers"]

In [51]:
#add suffix for column titles
suff = "_per_40"
for_40 = [stat+suff for stat in per_40]

In [52]:
#zip two lists together
combined_40 = list(zip(per_40,for_40))

In [211]:
#check combined list
combined_40

[('assists', 'assists_per_40'),
 ('blocks', 'blocks_per_40'),
 ('points', 'points_per_40'),
 ('steals', 'steals_per_40'),
 ('three_pointers', 'three_pointers_per_40'),
 ('total_rebounds', 'total_rebounds_per_40'),
 ('turnovers', 'turnovers_per_40')]

In [212]:
#run for loop to create per_40 columns
for stat in combined_40:
    ncaa2_df[stat[1]] = (ncaa2_df[stat[0]] /ncaa2_df["minutes_played"])*40

In [213]:
#fill nan's with column mean
ncaa2_df.fillna(ncaa2_df.mean(), inplace =True)

In [214]:
#convert height to inches

height = [ guy.split('-') for guy in ncaa2_df["height"]]
inches = [(int(dim[0])*12) + int(dim[1]) for dim in height]

In [215]:
ncaa2_df['height'] = inches

In [216]:
#check for duplicates
print(len(ncaa2_df))
duplicate_id = ncaa2_df.duplicated(subset=['player_id'])
if duplicate_id.any():
    print(ncaa2_df.loc[duplicate_id])

853


In [30]:
def check_dupes(df):
    duplicate_id = df.duplicated(subset=['player_id'])
    if duplicate_id.any():
        print(df.loc[duplicate_id])

In [237]:
check_dupes(combined_df)

In [217]:
combined_df = pd.merge(clean_df, ncaa2_df, on='player_id', how='inner')
len(combined_df)

853

In [222]:
ncaa2_df.head()

,player_id,name,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40,college,class
0,kenyon-martin-1,Kenyon Martin,142,292,0.587,0.586,0.494,0.581,116,81,2042.0,1279,124,0.222000,2.0,869,0.592,14.3,139.0,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821,cincinnati,Player
1,aj-guyton-1,A.J. Guyton,403,52,0.541,0.455,0.249,0.790,128,73,3260.0,2100,129,0.414000,283.0,429,0.572,13.6,218.0,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847,indiana,Player
2,jake-voskuhl-1,Jake Voskuhl,124,193,0.542,0.542,0.555,0.656,138,83,2184.0,863,86,0.329162,0.0,880,0.573,23.0,160.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403,uconn,Player
3,khalid-el-amin-1,Khalid El-Amin,479,10,0.486,0.416,0.257,0.822,108,70,2149.0,1650,186,0.355000,195.0,319,0.527,15.6,188.0,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302,uconn,Bust
4,mike-smith-1,Mike Smith,147,42,0.507,0.428,0.357,0.769,55,80,1938.0,1062,90,0.318000,130.0,449,0.551,20.0,241.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200,university of louisiana at monroe,Bust


In [227]:
#add class and college columns to NCAA
college = clean_df["college"]
classes = clean_df["class"]

ncaa2_df.insert(loc=2, column='class', value=classes)
ncaa2_df.insert(loc=3, column='college', value=college)

In [57]:
#export NCAA data to csv
ncaa2_df.to_csv("NCAA_data.csv",index = False, header = True)

NameError: name 'ncaa2_df' is not defined

In [232]:
#export NBA data to csv
clean_df.to_csv("NBA_cleaned.csv",index = False, header = True)

In [233]:
#export combined data to csv
combined_df.to_csv("combined_data.csv",index = False, header = True)

In [ ]:
id_df = ncaa2_df[["player_id", "name"]]
id_df.rename(columns = {'name':'Player'}, inplace = True) 

In [4]:
#pull in 2020 NBA Draft prospects
df_2020 = pd.read_csv("2020_draft_data/2020_prospects.csv")


In [5]:
# Get rows where year == intl.
intl = df_2020[ df_2020['Year'] == "Intl." ].index

# Delete these row indexes from dataFrame
df_2020.drop(intl, inplace=True)

In [6]:
#re-index df
df_2020.reset_index(drop=True, inplace=True)
df_2020.head()

,Rank,Name,School,Year
0,1,Anthony Edwards,Georgia,Fr.
1,2,James Wiseman,Memphis,Fr.
2,4,Obi Toppin,Dayton,So.
3,5,Jaden McDaniels,Washington,Fr.
4,6,Nico Mannion,Arizona,Fr.


In [9]:
make_ids(df_2020['Name'])

['anthony-edwards-1',
 'james-wiseman-1',
 'obi-toppin-1',
 'jaden-mcdaniels-1',
 'nico-mannion-1',
 'daniel-oturu-1',
 'cole-anthony-1',
 'vernon-carey-1',
 'tyrese-haliburton-1',
 'onyeka-okongwu-1']

In [12]:
#see which players are generating errors when trying to call college data 
for player in tqdm_notebook(makes_ids):
    try:
        college = Player(player)
    except:
        print(f'Error for {player} in index # {makes_ids.index(player)}')
        

Error for obi-toppin-1 in index # 2
Error for vernon-carey-1 in index # 7
Error for josh-green-1 in index # 11
Error for jay-scrubb-1 in index # 28
Error for nick-richards-1 in index # 31
Error for derrick-alston-jr-1 in index # 42
Error for cam-mack-1 in index # 51



obadiah-toppin-1
vernon-careyjr-1
josh-green-2
remove jay scrubb
nick-richards-2
derrick-alston-2
cameron-mack-1

In [13]:
replace_2020 = [('obadiah-toppin-1', 2), ('vernon-careyjr-1', 7), ('josh-green-2', 11), ('nick-richards-2', 31), 
          ('derrick-alston-2', 42),('cameron-mack-1', 51)]

In [15]:
#replace incorrect id's
for player in replace_2020:
    makes_ids[player[1]] = player[0]

In [16]:
#remove players not in NCAA
df_2020 = df_2020.drop([28])
makes_ids.pop(28)

'jay-scrubb-1'

In [17]:

#re-index clean_df
df_2020.reset_index(drop=True, inplace=True)

In [18]:
df_2020['Player_id'] = makes_ids 

In [19]:
#define function to get player_id colleges
def college_try(dataframe):
    global college_list
    college_list = []
    for index, row in tqdm_notebook(dataframe.iterrows()):
        try:
            college =  Player(row["Player_id"]).team_abbreviation
            college_list.append(college)
        except:
            print('Error for', row["Player_id"], 'at index #', index)


In [20]:
college_try(df_2020)

Error for reggie-perry-1 at index # 22



In [21]:
#insert correct value for player #22
makes_ids[22] = 'reggie-perry-2' 

In [22]:
df_2020['Player_id'] = makes_ids

In [23]:
#run college_try again
college_try(df_2020)

In [24]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [25]:
#add to  df
df_2020['College_id'] = college_list

In [28]:
college_abv(df_2020)

In [33]:
#create word column used for college matching
word = [words.split()[0] for words in df_2020['College_id']]
df_2020['word'] = word

In [34]:
#convert School column to lower case
df_2020['School'] = df_2020['School'].str.lower()

df_2020.head()

,Rank,Name,School,Year,Player_id,College_id,word
0,1,Anthony Edwards,georgia,Fr.,anthony-edwards-1,missouri state,missouri
1,2,James Wiseman,memphis,Fr.,james-wiseman-1,memphis,memphis
2,4,Obi Toppin,dayton,So.,obadiah-toppin-1,dayton,dayton
3,5,Jaden McDaniels,washington,Fr.,jaden-mcdaniels-1,washington,washington
4,6,Nico Mannion,arizona,Fr.,nico-mannion-1,arizona,arizona


In [35]:
wrong_players = []
for index, row in df_2020.iterrows():
    if search(row["word"], row["School"]):
        print(row["Name"], "at ", index, " matches")
    else:
        print(row["Name"], "at ", index, " does NOT match")
        wrong_players.append((row["Name"], index))

Anthony Edwards at  0  does NOT match
James Wiseman at  1  matches
Obi Toppin at  2  matches
Jaden McDaniels at  3  matches
Nico Mannion at  4  matches
Daniel Oturu at  5  matches
Cole Anthony at  6  matches
Vernon Carey at  7  matches
Tyrese Haliburton at  8  matches
Onyeka Okongwu at  9  does NOT match
Isaac Okoro at  10  matches
Josh Green at  11  matches
Zeke Nnaji at  12  matches
Isaiah Stewart at  13  does NOT match
Ashton Hagans at  14  matches
Precious Achiuwa at  15  matches
Aaron Nesmith at  16  matches
Tyrese Maxey at  17  matches
Jordan Nwora at  18  matches
Ochai Agbaji at  19  matches
Paul Reed at  20  does NOT match
Jalen Smith at  21  matches
Reggie Perry at  22  matches
Devon Dotson at  23  matches
Udoka Azubuike at  24  matches
CJ Elleby at  25  matches
Landers Nolley at  26  does NOT match
Devin Vassell at  27  matches
Cassius Stanley at  28  matches
Saddiq Bey at  29  matches
Nick Richards at  30  matches
Elijah Hughes at  31  matches
Tyler Bey at  32  matches
Chris

In [36]:
wrong_players

[('Anthony Edwards', 0),
 ('Onyeka Okongwu', 9),
 ('Isaiah Stewart', 13),
 ('Paul Reed', 20),
 ('Landers Nolley', 26),
 ('Chris Smith', 33),
 ('Malik Fitts', 42),
 ('Desmond Bane', 49)]

### players to replace
anthony-edwards-2
isaiah-stewart-2
paul-reed-5
landers-nolleyii-1
chris-smith-19

In [37]:
update_2020 = [('anthony-edwards-1', 'anthony-edwards-2'), ('isaiah-stewart-1','isaiah-stewart-2'),
               ('paul-reed-1', 'paul-reed-5'),('landers-nolley-1', 'landers-nolleyii-1'),
               ('chris-smith-19', 'chris-smith-19')]

In [38]:
#update player ids 
for player in update_2020:
    df_2020.replace(to_replace = player[0],
                    value = player[1], inplace = True)

In [41]:
#insert player stats into ncaa_data
ncaa_data = []
for player in tqdm_notebook(df_2020["Player_id"]):
    ncaa = Player(player)
    player_list = []
    player_dict = {}
    for stat in stats:
        try:
            var = eval(f'ncaa.{stat}')
            player_list.append((stat, var))
        except:
            var = np.nan
            print(player,stat, var)
            player_list.append((stat, var))
    player_dict.update(player_list)
    ncaa_data.append(player_dict)

In [43]:
#convert ncaa_data to df
draft_df = pd.DataFrame(ncaa_data)


In [46]:
#add college, rank and year to ncaa_df
draft_df["college"] = df_2020["College_id"]
draft_df["rank"] = df_2020["Rank"]
draft_df["year"] = df_2020["Year"]


In [47]:
#define new df with desired columnes
draft2020_df = draft_df[["rank", "player_id", "name", "college", "year", "assists", "blocks", "effective_field_goal_percentage",
                    "field_goal_percentage", "free_throw_attempt_rate","free_throw_percentage", "games_played", "height",
                    "minutes_played", "points","steals", "three_point_percentage", "three_pointers", "total_rebounds",
                    "true_shooting_percentage", "turnover_percentage", "turnovers", "win_shares_per_40_minutes" ]]

In [48]:
#fill_na's
draft2020_df.fillna(draft2020_df.mean(), inplace=True)

In [53]:
#add per 40 stats
for stat in combined_40:
    draft2020_df[stat[1]] = (draft2020_df[stat[0]] /draft2020_df["minutes_played"])*40

In [54]:
#calculate height in inches
height = [ guy.split('-') for guy in draft2020_df["height"]]
inches = [(int(dim[0])*12) + int(dim[1]) for dim in height]
draft2020_df['height'] = inches

In [55]:
#export to csv
draft2020_df.to_csv("Draft_data.csv",index = False, header = True)